<a href="https://colab.research.google.com/github/akingno/colab_temp/blob/main/Aloe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers accelerate



Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "HPAI-BSC/Llama3.1-Aloe-Beta-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!git clone https://github.com/pubmedqa/pubmedqa.git

Cloning into 'pubmedqa'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 40 (delta 0), reused 1 (delta 0), pack-reused 37 (from 1)
Receiving objects: 100% (40/40), 704.87 KiB | 1.89 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [ ]:
cd pubmedqa

/content/pubmedqa


In [ ]:
cd preprocess

/content/pubmedqa/preprocess


In [ ]:
!python split_dataset.py pqal

In [ ]:
import json
from google.colab import files

# 读取数据集
with open("/content/pubmedqa/data/test_set.json", "r") as f:
    data = json.load(f)

results = []


keys = list(data.keys())[40:50]

for key in keys:
    input_text = data[key]["QUESTION"]  # 获取问题
    data_answer_text = data[key]["LONG_ANSWER"]  # 获取标准答案

    print(f"问题 (Key: {key}): {input_text}")
    print(f"标准答案: {data_answer_text}")

    # 创建模型输入，加入问题和“回答”提示
    prompt = f"Question：{input_text}\n Answer："

    # 输入模型
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # 生成答案
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

    # 解码生成的答案
    generated_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 打印生成的回答和标准答案
    print(f"生成回答: {generated_answer}")
    print("-" * 50)  # 分隔符

    # 将结果添加到列表
    results.append({
        "key": key,
        "question": input_text,
        "standard_answer": data_answer_text,
        "generated_answer": generated_answer
    })

# 将结果保存到本地 JSON 文件
output_file = "/content/generated_results.json"
with open(output_file, "w") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

# 下载保存的 JSON 文件
files.download(output_file)


问题 (Key: 10158597): Does a dedicated discharge coordinator improve the quality of hospital discharge?
标准答案: The introduction of a discharge coordinator improved the quality of discharge planning, but at additional cost.
生成回答: Question：Does a dedicated discharge coordinator improve the quality of hospital discharge?
 Answer：Yes, a dedicated discharge coordinator can improve the quality of hospital discharge. Here's why:

1. **Streamlined Discharge Process**: A dedicated discharge coordinator can facilitate a smoother and more efficient discharge process, reducing delays and complications. They ensure that all necessary steps, such as arranging follow-up appointments, home health services, and medication management, are completed before the patient leaves the hospital.
2. **Improved Patient Safety**: By coordinating the discharge process, the risk of adverse events, such as medication errors or hospital readmissions, can be minimized. The coordinator can verify that the patient has a saf

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>